# `giotto-tda` persistent homology tutorial

# 2. Tutorial: Hilbert embedding for gravitational waves 

Gravitational waves (GW) are caused by many cosmic events, such as supernovaes detection, colliding black holes, or neutron stars. While challenging due to their small amplitude, identifying GW is considered as [opening a new window on the universe](https://www.ligo.caltech.edu/page/why-detect-gw). In the literature, formally, the task is often presented as a binary classification problem, which consists of deciding whether a signal contains the signature of a GW or not. Effective approaches with CNNs have been proposed, which, however, require significant data to be trained. 

In [an article](https://arxiv.org/abs/1910.08245), Chrisopher Bresten and Jae-Hun Jung proposed to include topological signatures in a CNN classifier to improve the models' performance. In this tutorial, we present an example of a classification pipeline relying only on topological features.

With giotto-tda, we can reproduce part of the research and address the following question. 

**Research question:** Would the Hilbert transform (a signal processing approach) help?

## 2.1 Data

In [ ]:
from data.generate_datasets import make_gravitational_waves
from pathlib import Path

R = 0.65
n_signals = 200
DATA = Path("./data")

noisy_signals, gw_signals, labels = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, r_min=R, r_max=R, n_snr_values=1
)

print(f"Number of noisy signals: {len(noisy_signals)}")
print(f"Number of timesteps per series: {len(noisy_signals[0])}")

Next let's visualise the two different types of time series that we wish to classify: one that is pure noise vs. one that is composed of noise plus an embedded gravitational wave signal:

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# get the index corresponding to the first pure noise time series
background_idx = np.argmin(labels)
# get the index corresponding to the first noise + gravitational wave time series
signal_idx = np.argmax(labels)

ts_noise = noisy_signals[background_idx]
ts_background = noisy_signals[signal_idx]
ts_signal = gw_signals[signal_idx]

fig = make_subplots(rows=1, cols=2, subplot_titles=("Noise", "Noise and GW"))

fig.add_trace(
    go.Scatter(x=list(range(len(ts_noise))), y=ts_noise, mode="lines", name="noise"),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(ts_background))),
        y=ts_background,
        mode="lines",
        name="background",
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_signal))), y=ts_signal, mode="lines", name="signal"),
    row=1,
    col=2,
)
fig.show()

We consider a setting simpler than in the article. We create a dataset as follows:

* Generate gravitational wave signals that correspond to non-spinning binary black hole mergers
* Generate a noisy time series and embed a gravitational wave signal with probability 0.5 at a random time.

The result is a set of time series of the form

$$ s = g + \epsilon \frac{1}{R}\xi $$

where $g$ is a gravitational wave signal from the reference set, $\xi$ is Gaussian noise, $\epsilon=10^{-19}$ scales the noise amplitude to the signal, and $R$ is a parameter that controls the signal-to-noise-ratio (SNR).

Here, we set $R$ to $0.65$, what gives an SNR of 17.89, placing ourselves in a favorable setting, compared to the article.

## 2.2 Recurrent structure with Taken's Embedding
Embeddings allow you to study the recurrent structure, present in a time series.
![A 2-dimensional time delay embedding](images/time_delay_embedding.gif)
$$
TD_{d,\tau} s : \mathbb{R} \to \mathbb{R}^{d}\,, \qquad t \to \begin{bmatrix}
           s(t) \\
           s(t + \tau) \\
           s(t + 2\tau) \\
           \vdots \\
           s(t + (d-1)\tau)
         \end{bmatrix},
$$

More details about the Taken's embedding in the context of TDA can be found in one of the review works [Topological Time Series Analysis](http://www.ams.org/notices/201905/rnoti-p686.pdf) or in the tutorial.

In [ ]:
from gtda.time_series import SingleTakensEmbedding

embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(parameters_type="search", n_jobs=6,
                                 time_delay=embedding_time_delay, dimension=embedding_dimension,
                                 stride=stride)

y_gw_embedded = embedder.fit_transform(gw_signals[0]) # argument: a single time series

We can use PCA to project our high-dimensional space to 3-dimensions for visualisation:

In [ ]:
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud

pca = PCA(n_components=3)
y_gw_embedded_pca = pca.fit_transform(y_gw_embedded)

plot_point_cloud(y_gw_embedded_pca)

From the plot we can see that the decaying periodic signal generated by a black hole merger emerges as a _spiral_ in the time delay embedding space! For contrast, let's compare this to one of the pure noise time series in our sample:

In [ ]:
y_noise_embedded = embedder.fit_transform(noisy_signals[background_idx])
y_noise_embedded_pca = pca.fit_transform(y_noise_embedded)

plot_point_cloud(y_noise_embedded_pca)

Evidently, pure noise resembles a high-dimensional ball in the time delay embedding space.

Clearly, there is a difference in the global structure of the data. Let us see if it is captured by persistent homology. Let's build a pipeline.

## 2.3 Classification pipeline

In [ ]:
from gtda.time_series import TakensEmbedding

from gtda.metaestimators import CollectionTransformer
from gtda.pipeline import Pipeline

takens = TakensEmbedding(time_delay=200,
                         dimension=10,
                         stride=10) # argument: collection of time series

batch_pca = CollectionTransformer(PCA(n_components=3), n_jobs=-1) # n_jobs

takens_pipeline = Pipeline(steps=[("takens", takens),
                                  ("pca", batch_pca)])

In [ ]:
from gtda.diagrams import PersistenceEntropy, Scaler
from gtda.homology import WeakAlphaPersistence

from sklearn.linear_model import LogisticRegression

weak_alpha = WeakAlphaPersistence(homology_dimensions=[0, 1])

scaling = Scaler()

entropy = PersistenceEntropy(normalize=True,)

logistic = LogisticRegression()

In [ ]:
steps = [("embedder", takens_pipeline),
         ("persistence", weak_alpha),
         ("scaling", scaling),
         ("entropy", entropy),
         ("logistic", logistic)]

Think ahead: cross validation, so caching!

In [ ]:
topological_classifier = Pipeline(steps)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    noisy_signals, labels, test_size=0.3, random_state=42
)

In [ ]:
topological_classifier.fit(X_train, y_train)

In [ ]:
from utils import print_scores
print_scores(topological_classifier, X_train, X_valid, y_train, y_valid, show_auc=True)

In [ ]:
topological_classifier[-1].coef_

The coefficients in the regression are negative, indicating that the distribution of persistence values for GW is less "uniform".



## 2.4 "New" idea: Hilbert transform map
In signal processing, the *analytic signal representation* $H(s)(t)$ of the signal $s(t)$
$$H(s)(t) = \frac{1}{\pi}p.v.\int_{-\infty}^{\infty} \frac{s(\tau)}{t-\tau}d\tau,$$
is a common denominator for many spectral methods and it often captures the recurrent nature of a system.
In [an article](https://ieeexplore-ieee-org.proxy.scd.u-psud.fr/document/8553502), it has been proposed as a generalization of the Takens embedding.

In [ ]:
from scipy.signal import hilbert

def hilbert_map(x):
    analytic_signal = hilbert(x)
    return np.stack([np.real(analytic_signal), np.imag(analytic_signal)], axis=-1)

In [ ]:
from sklearn.preprocessing import FunctionTransformer

hilbert_embedding = CollectionTransformer(FunctionTransformer(func=hilbert_map))

Format of the input (why it is correct to use a function transformer ()) Or docstring for the function

Let's look at an example of the Hilbert map

In [ ]:
theta = np.linspace(0, 4*np.pi, 300)
signal = np.sin(theta*2.05) + np.cos(theta*4)
hilbert_signal = hilbert_embedding.fit_transform([signal])[0]

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Signal", "Analytic signal"))

fig.add_trace(
    go.Scatter(x=theta, y=signal, mode="lines", name="noise"),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(x=hilbert_signal[:,0], y=hilbert_signal[:,1], mode="lines", name="signal"),
    row=1,
    col=2,
)
fig.show()

What does it do on our point cloud?

In [ ]:
y_gw_hilbert = hilbert_embedding.fit_transform(gw_signals)
y_gw_hilbert.shape

In [ ]:
plot_point_cloud(y_gw_hilbert[0,])

The result is similar to the PCA of the TakensEmbeddnig. However, there are more circles on the outside.

In [ ]:
y_noise_hilbert = hilbert_embedding.transform(noisy_signals)
plot_point_cloud(y_noise_hilbert[background_idx, ::5])

In [ ]:
dgms = weak_alpha.fit_transform([y_gw_hilbert[0, ], y_noise_hilbert[background_idx, ]])
dgm_gw, dgm_noise = Scaler().fit_transform(dgms)

In [ ]:
weak_alpha.plot([dgm_gw])

In [ ]:
weak_alpha.plot([dgm_noise])

There are more highly-persistent points in dimension 1.

## Compare the two models

In [ ]:
param_grid = [{"embedder": [takens_pipeline],
               #"embedder__takens__dimension": [10, ...],
               #"embedder__takens__time_delay" : [200, ...],
              },
              {"embedder": [hilbert_embedding]}]

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

grid_search_results = GridSearchCV(topological_classifier,
                                   param_grid=param_grid,
                                   cv=KFold(n_splits=3, shuffle=True),
                                   scoring="roc_auc",
                                   return_train_score=True,
                                  ).fit(X_train, y_train)

In [ ]:
best_pipeline = grid_search_results.best_estimator_
print(f"The best score is {grid_search_results.best_score_}, achieved by:\n{best_pipeline}")

In [ ]:
grid_search_results.cv_results_["mean_test_score"]

## Further ideas

1. Cross-validate the parameters of the `TakensEmbedding`

2. Construct a graph from the time series: the Hilbert map is not the only idea different from the standard Takens Embedding. For example, [(A. Myers & al.)](https://arxiv.org/pdf/1904.07403.pdf) authors have proposed the permutation embedding, and construction a graph.

3. Use other features - landscapes, persistence images, silhouettes.